
# Analyzing 10-K Reports using AgentRearrange with Mixed Sequential and Parallel Swarm

This notebook will guide you through the process of setting up a mixed sequential and parallel workflow to analyze 10-K reports filed with the SEC. 
Using the `swarms` framework and the `AgentRearrange` class, we'll create specialized agents that focus on different aspects of the 10-K reports, 
such as financial statements, management discussion, and risk factors. The workflow will process some tasks sequentially while others will run in parallel.

For more detailed information, please refer to the [Swarms Documentation](https://docs.swarms.world/en/latest/).


# Installation
Ensure you have swarms and other packages installed with:

```bash
$ pip3 install -U swarms python-dotenv PyPDF2
```



## Step 1: Setting Up the Environment

First, we need to load our environment variables and initialize the model. Make sure you have your OpenAI API key stored 
in a `.env` file in the same directory as this notebook.


In [ ]:

import os
from dotenv import load_dotenv
from swarms import OpenAIChat

# Load environment variables
load_dotenv()

# Get the API key from the environment variables
api_key = os.environ.get("OPENAI_API_KEY")

# Initialize the OpenAIChat model
llm = OpenAIChat(
    temperature=0.5, 
    model_name="gpt-4o-mini", 
    openai_api_key=api_key, 
    max_tokens=4000
)

print("Environment set up and model initialized successfully!")



## Step 2: Creating the Agents

Next, we'll create the agents that will analyze different sections of the 10-K report. Each agent is assigned a specific task 
with a detailed system prompt. The agents include:
- **FinancialStatementsExpert**: Analyzes the financial statements section.
- **ManagementDiscussionReviewer**: Reviews the management's discussion and analysis.
- **RiskAssessmentAnalyst**: Examines the risk factors section.

Let's create each agent with its specific prompt.


In [ ]:

from swarms import Agent

# Initialize FinancialStatementsExpert agent
financial_statements_expert = Agent(
    agent_name="FinancialStatementsExpert",
    llm=llm,
    max_loops=1,
    autosave=True,
    system_prompt=(
        "You are an expert in financial analysis. Your task is to thoroughly analyze the financial statements section "
        "of the 10-K report. Pay close attention to the balance sheet, income statement, and cash flow statement. Identify "
        "key financial metrics such as revenue growth, profit margins, debt levels, and cash flow trends. Provide a detailed "
        "summary of your findings, highlighting any potential red flags or areas of concern."
    )
)

# Initialize ManagementDiscussionReviewer agent
management_discussion_reviewer = Agent(
    agent_name="ManagementDiscussionReviewer",
    llm=llm,
    max_loops=1,
    autosave=True,
    system_prompt=(
        "You specialize in understanding and interpreting management's discussion and analysis. Your task is to critically "
        "review this section of the 10-K report. Identify the key themes discussed by management, such as company performance, "
        "strategic initiatives, and forward-looking statements. Analyze the tone and language used to assess management's confidence "
        "in the company's future prospects. Summarize the key points and provide insights into potential risks or opportunities."
    )
)

# Initialize RiskAssessmentAnalyst agent
risk_assessment_analyst = Agent(
    agent_name="RiskAssessmentAnalyst",
    llm=llm,
    max_loops=1,
    autosave=True,
    system_prompt=(
        "As an expert in risk management, your task is to analyze the risk factors section of the 10-K report. Carefully read through "
        "the risks identified by the company, and assess their potential impact on the company's operations and financial performance. "
        "Prioritize the risks based on their severity and likelihood, and provide recommendations on how the company might mitigate these risks."
    )
)

# Initialize SummaryReportGenerator agent
summary_report_generator = Agent(
    agent_name="SummaryReportGenerator",
    llm=llm,
    max_loops=1,
    autosave=True,
    dashboard=False,
    system_prompt=(
        "You are a specialist in creating executive summaries. Your task is to consolidate the findings from the financial statements, "
        "management discussion, and risk factors into a cohesive summary report. Highlight the most important insights and provide "
        "a clear and concise overview that can be presented to senior management."
    )
)

print("Agents created successfully!")



## Step 3: Defining the Workflow with AgentRearrange

Now that we have our agents, we'll set up a workflow using the `AgentRearrange` class. This workflow will process the tasks 
sequentially and in parallel, as defined by the flow: `"FinancialStatementsExpert -> ManagementDiscussionReviewer, RiskAssessmentAnalyst, SummaryReportGenerator"`.


In [ ]:

from swarms import AgentRearrange

# Create a list of agents
agents = [financial_statements_expert, management_discussion_reviewer, risk_assessment_analyst, summary_report_generator]

# Define the flow pattern
flow = "FinancialStatementsExpert -> ManagementDiscussionReviewer, RiskAssessmentAnalyst, SummaryReportGenerator"

# Using AgentRearrange class to create the workflow
agent_system = AgentRearrange(agents=agents, flow=flow, max_loops=1)

print("Workflow defined successfully!")



## Step 4: Running the Workflow

Now it's time to execute the workflow. The agents will analyze the assigned sections of the 10-K report, following the defined 
flow where the financial statements are analyzed first, followed by parallel analyses of the management discussion, risk factors, 
and then generating the summary report.

First, we're going to parse and load the PDF into the swarm, and then the pdf is going to be sent to all of the agents at once.

In [ ]:

# pip install PyPDF2 
import PyPDF2

def load_pdf_to_string(file_path: str):
    # Open the PDF file in read-binary mode
    with open(file_path, 'rb') as file:
        # Create a PDF file reader object
        pdf_reader = PyPDF2.PdfFileReader(file)
        
        text = ""
        # Loop through all the pages (if multiple) and extract text
        for page_number in range(pdf_reader.getNumPages()):
            page = pdf_reader.getPage(page_number)
            text += page.extractText()
        
        # Return the extracted text
        return text

ten_k_data = load_pdf_to_string("sample_10k.pdf")

# Run the workflow
output = agent_system.run(f"Analyze the 10-K report and generate a summary report: {ten_k_data}")
print(output)



## Conclusion and Next Steps

You've successfully set up and run a mixed sequential and parallel workflow to analyze 10-K reports using specialized agents. 
This approach demonstrates how multi-agent systems can be used to automate complex financial analysis tasks, providing valuable 
insights from detailed reports.

### Next Steps:
- Explore the [Swarms Documentation](https://docs.swarms.world/en/latest/) for more details and advanced features.
- Experiment with different agents, tasks, and scenarios to see how the system adapts.
- Consider applying this framework to other types of financial reports or even different domains.

Thank you for following along!
